
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

file = pd.read_csv("data/Corona_NLP_train.csv", encoding='latin-1')
file = file[file['Sentiment'] != 'Neutral']

X = file['OriginalTweet'].values
Y = np.array([1 if sentiment in ["Positive", "Extremely Positive"] else 0 for sentiment in file['Sentiment']])

# Split the data into train (60%), test (15%), and validation (25%) sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.4, shuffle=False)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_temp, Y_temp, test_size=0.625, shuffle=False)

print(f"Train set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Validation set size: {len(X_valid)}")

Train set size: 20066
Test set size: 5016
Validation set size: 8362


2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**


In [3]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


# Define the pipeline
model = Pipeline([
    ('vectorizer', CountVectorizer()),  # Convert text to numerical features
    ('classifier', LogisticRegression(solver='liblinear', max_iter=1000))  # Logistic regression for classification
])

# Fit the pipeline on the training data
model.fit(X_train, Y_train)

# Test predict_proba functionality
proba = model.predict_proba(X_test[:5])
print("Predicted probabilities for the first 5 test samples:")
print(proba)

# Check predictions on validation data
predictions = model.predict(X_valid)
print("Predictions on the validation set:")
print(predictions[:10])



Predicted probabilities for the first 5 test samples:
[[0.87766879 0.12233121]
 [0.99712779 0.00287221]
 [0.03468536 0.96531464]
 [0.7988057  0.2011943 ]
 [0.43882767 0.56117233]]
Predictions on the validation set:
[1 0 1 1 0 0 0 0 1 0]


In [4]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

from sklearn.metrics import precision_score, recall_score

# Predict on the test set
Y_test_pred = model.predict(X_test)

# Calculate precision and recall for both classes
precision_0 = precision_score(Y_test, Y_test_pred, pos_label=0)  # Precision for negative class (0)
recall_0 = recall_score(Y_test, Y_test_pred, pos_label=0)        # Recall for negative class (0)

precision_1 = precision_score(Y_test, Y_test_pred, pos_label=1)  # Precision for positive class (1)
recall_1 = recall_score(Y_test, Y_test_pred, pos_label=1)        # Recall for positive class (1)


4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 


In [ ]:

# Part 4

def cost(model, threshold, X, Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    
    # Get the predicted probabilities
    
    probabilities = model.predict_proba(X)[:, 1]
    
    # Make predictions based on the threshold
    predictions = (probabilities >= threshold).astype(int)
    
    # Calculate the cost
    cost = np.mean((1 - predictions) * Y + 5 * (1 - Y) * predictions)
    
    return cost
print("cost: " + str(cost(model, 0.3, X, Y))) # Average cost 2.167 for threshold 0.5, 

cost: 0.38120440138739387


In [6]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

thresholds = np.linspace(0, 1, 101)  # Generate 101 thresholds from 0 to 1
costs = [cost(model, t, X_test, Y_test) for t in thresholds]

optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = min(costs)

print(f"Optimal threshold: {optimal_threshold}")
print(f"Cost at optimal threshold: {cost_at_optimal_threshold}")

Optimal threshold: 0.8
Cost at optimal threshold: 0.2727272727272727


[4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.

In [9]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
print(f"optimal cost {cost_at_optimal_threshold_valid}")
deviation = np.sqrt((5**2*np.log(200))/(2*len(X_valid)))
print(f"epsilon {deviation}")
cost_interval_valid = (cost_at_optimal_threshold_valid-deviation, cost_at_optimal_threshold_valid+deviation)
print("99% confidence interval for cost:" + str(cost_interval_valid))
n_valid = len(Y_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

optimal cost 0.27744558718010043
epsilon 0.08899567661998982
99% confidence interval for cost:(np.float64(0.18844991056011062), np.float64(0.36644126380009023))


Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [8]:

# Part 7

# Define the cost function C
def calculate_cost_C(model, threshold, X, Y):
    probabilities = model.predict_proba(X)[:, 1]  # Get probabilities for the positive class
    C = (1 - (probabilities >= threshold).astype(int)) * Y + \
        5 * (1 - Y) * (probabilities >= threshold).astype(int)
    return C

# Calculate C for the validation set
C_values = calculate_cost_C(model, optimal_threshold, X_valid, Y_valid)

# Calculate the empirical variance of C
variance_of_C = np.var(C_values, ddof=1)  # Use ddof=1 for sample variance

# Bennett's inequality to calculate the confidence interval
n = len(C_values)
delta = 0.01  # 99% confidence
epsilon = np.sqrt((2 * variance_of_C * np.log(1 / delta)) / n)

# Mean of C
mean_of_C = np.mean(C_values)

# Confidence interval
interval_of_C = (mean_of_C - epsilon, mean_of_C + epsilon)


assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)